In [1]:
import math
import pandas as pd

from ipywidgets import widgets

# Pointsource calc

## Vstupní parametry

* Výška pointsource (m) - `pointsource_height`
* Downtilt úhel (°) - `downtilt_angle`
* Vertikální vyzařovací úhel (°) - `vertical_dispersion_angle`
* Výška posluchače (m) - `listeners_height`

In [2]:
pointsource_height = 3 #m
downtilt_angle = 5 #deg
vertical_dispersion_angle = 60 #deg - ETX-12P
listeners_height = 1.7 #m

## Implementace

In [3]:
def convert_deg_to_rad( deg : float ) -> float:
    angle = deg
    if ( deg >= 90 ):
        angle_above = angle - 90
        angle = 90 - angle_above
        
    
    value = ( ( ( angle % 90 )   / 360 ) * 2 * math.pi )
    return value

def calculate_onax_point( listeners : bool, pointsource_height : float, downtilt_angle : float, listeners_height : float ) -> float:
    offset = 0.0
    if ( listeners == True ):
        offset = listeners_height
    else:
        offset = 0
    
    distance_to_onax_point = ( pointsource_height - offset ) / math.tan( convert_deg_to_rad( downtilt_angle ) )
    
    return distance_to_onax_point

def calculate_ofax_near_point( listeners : bool, pointsource_height : float, downtilt_angle : float, vertical_dispersion_angle : float, listeners_height : float ) -> float:
    offset = 0.0
    if ( listeners == True ):
        offset = listeners_height
    else:
        offset = 0
    
    angle =  180 - ( ( vertical_dispersion_angle / 2 ) + downtilt_angle )
    distance_to_ofax_near_point = ( pointsource_height - offset ) / math.tan( convert_deg_to_rad( angle ) ) 
    
    return distance_to_ofax_near_point

def calculate_ofax_far_point( listeners : bool, pointsource_height : float, downtilt_angle : float, vertical_dispersion_angle : float, listeners_height : float ) -> float:
    offset = 0.0
    if ( listeners == True ):
        offset = listeners_height
    else:
        offset = 0
    
    distance_to_ofax_near_point = ( pointsource_height - offset ) / math.tan( convert_deg_to_rad( downtilt_angle - vertical_dispersion_angle / 2 ) )
    
    return distance_to_ofax_near_point

In [4]:
onax_gnd_point = calculate_onax_point(
    listeners=False,
    pointsource_height=pointsource_height,
    downtilt_angle=downtilt_angle,
    listeners_height=listeners_height
)
onax_listeners_point = calculate_onax_point(
    listeners=True,
    pointsource_height=pointsource_height,
    downtilt_angle=downtilt_angle,
    listeners_height=listeners_height
)

In [5]:
ofax_near_gnd_point = calculate_ofax_near_point(
    listeners=False,
    pointsource_height=pointsource_height,
    downtilt_angle=downtilt_angle,
    vertical_dispersion_angle=vertical_dispersion_angle,
    listeners_height=listeners_height
)

ofax_near_listeners_point = calculate_ofax_near_point(
    listeners=True,
    pointsource_height=pointsource_height,
    downtilt_angle=downtilt_angle,
    vertical_dispersion_angle=vertical_dispersion_angle,
    listeners_height=listeners_height
)

In [6]:
ofax_far_gnd_point = calculate_ofax_far_point(
    listeners=False,
    pointsource_height=pointsource_height,
    downtilt_angle=downtilt_angle,
    vertical_dispersion_angle=vertical_dispersion_angle,
    listeners_height=listeners_height
)

ofax_far_listeners_point = calculate_ofax_far_point(
    listeners=True,
    pointsource_height=pointsource_height,
    downtilt_angle=downtilt_angle,
    vertical_dispersion_angle=vertical_dispersion_angle,
    listeners_height=listeners_height
)

## Porovnání více situací

In [7]:
downtilt_angle_list = [0.0,5.0, 15.0, 33.0]

downtilt_angle_dict = dict()

def calculate_characteristics( pointsource_height : float, downtilt_angle : float, vertical_dispersion_angle : float, listeners_height : float ) -> dict:
    return_dict = dict()
    
    ofax_near_listeners_point = calculate_ofax_near_point(
        listeners=True,
        pointsource_height=pointsource_height,
        downtilt_angle=downtilt_angle,
        vertical_dispersion_angle=vertical_dispersion_angle,
        listeners_height=listeners_height
    )
    
    ofax_near_gnd_point = calculate_ofax_near_point(
        listeners=False,
        pointsource_height=pointsource_height,
        downtilt_angle=downtilt_angle,
        vertical_dispersion_angle=vertical_dispersion_angle,
        listeners_height=listeners_height
    )
    
    return_dict["OFAX near - Listeners [m]"] = ofax_near_listeners_point
    return_dict["OFAX near - Ground [m]"] = ofax_near_gnd_point
    
    if ( downtilt_angle == 0 ):
        return_dict["ONAX - Listeners [m]"] = None
        return_dict["ONAX - Ground [m]"] = None
        return_dict["OFAX far - Listeners [m]"] = None
        return_dict["OFAX far - Ground [m]"] = None
        return return_dict
    
    
    onax_near_listeners_point = calculate_onax_point(
        listeners=True,
        pointsource_height=pointsource_height,
        downtilt_angle=downtilt_angle,
        listeners_height=listeners_height
    )
    
    onax_near_gnd_point = calculate_onax_point(
        listeners=False,
        pointsource_height=pointsource_height,
        downtilt_angle=downtilt_angle,
        listeners_height=listeners_height
    )
    
    return_dict["ONAX - Listeners [m]"] = onax_near_listeners_point
    return_dict["ONAX - Ground [m]"] = onax_near_gnd_point
    
    if ( vertical_dispersion_angle / 2 >= downtilt_angle ):
        return_dict["OFAX far - Listeners [m]"] = None
        return_dict["OFAX far - Ground [m]"] = None
        return return_dict
    
    ofax_far_listeners_point = calculate_ofax_far_point(
        listeners=True,
        pointsource_height=pointsource_height,
        downtilt_angle=downtilt_angle,
        vertical_dispersion_angle=vertical_dispersion_angle,
        listeners_height=listeners_height
    )
    
    ofax_far_gnd_point = calculate_ofax_far_point(
        listeners=False,
        pointsource_height=pointsource_height,
        downtilt_angle=downtilt_angle,
        vertical_dispersion_angle=vertical_dispersion_angle,
        listeners_height=listeners_height
    )
    
    return_dict["OFAX far - Listeners [m]"] = ofax_far_listeners_point
    return_dict["OFAX far - Ground [m]"] = ofax_far_gnd_point
    
    return return_dict

angle_dict = dict()

for angle in downtilt_angle_list:
    angle_dict[angle] = calculate_characteristics(
        pointsource_height=pointsource_height,
        downtilt_angle=angle,
        vertical_dispersion_angle=vertical_dispersion_angle,
        listeners_height=listeners_height
    )

In [8]:
angle_df = pd.DataFrame.from_dict(angle_dict)

In [9]:
angle_df

,0.0,5.0,15.0,33.0
OFAX near - Listeners [m],2.251666,1.856592,1.300000,0.662383
OFAX near - Ground [m],5.196152,4.284444,3.000000,1.528576
ONAX - Listeners [m],NaN,14.859068,4.851666,2.001824
ONAX - Ground [m],NaN,34.290157,11.196152,4.619595
OFAX far - Listeners [m],NaN,NaN,NaN,24.805478
OFAX far - Ground [m],NaN,NaN,NaN,57.243410


## Interaktivní verze

### ONAX aiming calculator

In [28]:
pointsource_height_input = widgets.FloatText( valuw=0.0, description="PS height:", disabled=False )
listeners_height_input = widgets.FloatText( value=1.7, description="Listen height:", disabled=False )
dispersion_angle_input = widgets.FloatText( value=0.0, description="Dispersion angle", disabled=False )
calculate_downtilt_checkbox = widgets.Checkbox( value=False, description="Calc aim", disabled=False )
downtilt_angle_input = widgets.FloatText( value=0.0, description="Downtilt angle", disabled=False )
onax_aiming_point_input = widgets.FloatText( value=0.0, description="Aiming dist", disabled=True )
calculate_button = widgets.Button( description="Calculate", disabled=False )
output_widget = widgets.Output()

def check_uncheck( change ):
    if ( calculate_downtilt_checkbox.value == True ):
        onax_aiming_point_input.disabled = False
        downtilt_angle_input.disabled = True
    else:
        onax_aiming_point_input.disabled = True
        downtilt_angle_input.disabled = False

calculate_downtilt_checkbox.observe( check_uncheck )

settings_v_box = widgets.VBox([pointsource_height_input, listeners_height_input, dispersion_angle_input, calculate_downtilt_checkbox, downtilt_angle_input, onax_aiming_point_input, calculate_button, output_widget])

In [31]:
def convert_rads_to_deg( rads : float ) -> float:
     return ( ( rads / ( math.pi / 2 ) ) * 90 )

def calculate_onax_aiming_angle( pointsource_height : float, listeners_height : float, onax_aiming_point : float, vertical_dispersion_angle : float ) -> float:
    angle_in_rads = math.atan( ( pointsource_height - listeners_height ) / onax_aiming_point )
    angle_in_deg = convert_rads_to_deg(angle_in_rads)
    return angle_in_deg

def calculate_aiming( change ):
    ps_height = float(pointsource_height_input.value)
    l_height = float(listeners_height_input.value)
    dispersion_angle = float(dispersion_angle_input.value)
    down_angle = float(downtilt_angle_input.value)
    aim_dist = float(onax_aiming_point_input.value)
    
    if ( calculate_downtilt_checkbox.value == True ):
        down_angle = calculate_onax_aiming_angle(
            pointsource_height=ps_height,
            listeners_height=l_height,
            onax_aiming_point=aim_dist,
            vertical_dispersion_angle=dispersion_angle
        )
        downtilt_angle_input.value = down_angle
    
    characteristics = calculate_characteristics(
        pointsource_height=ps_height,
        listeners_height=l_height,
        vertical_dispersion_angle=dispersion_angle,
        downtilt_angle=down_angle,
    )
    
    onax_aiming_point_input.value = characteristics["ONAX - Listeners [m]"]
    
    char_df = pd.DataFrame.from_dict(characteristics, orient="index")
    with( output_widget ):
        display(char_df, clear=True)

calculate_button.on_click( calculate_aiming )

### GUI

In [30]:
settings_v_box